In [2]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
import os
import re

print("--- Đang tải model và thông tin cần thiết ---")

model_path = os.path.join('/kaggle/input/input-model-sentiment/sentiment_analyzer_best_model.keras')
model_info_path = os.path.join('/kaggle/input/input-model-sentiment/sentiment_analyzer_model_info.pkl')

if os.path.exists(model_path):
    loaded_model = keras.models.load_model(model_path)
    print(f"Đã load model từ: {model_path}")
else:
    print(f"LỖI: Không tìm thấy model tại {model_path}")
    loaded_model = None

if os.path.exists(model_info_path):
    with open(model_info_path, 'rb') as f:
        model_info = pickle.load(f)
    print(f"Đã load model_info từ: {model_info_path}")
    
    MAX_LEN = model_info.get('max_len_sequence') 
    if MAX_LEN is None:
        print("LỖI: Không tìm thấy 'max_len_sequence' trong model_info. Đặt tạm là 128.")
        MAX_LEN = 128
    else:
        print(f"MAX_LEN được load từ model_info: {MAX_LEN}")

    label_encoder_filename = model_info.get('label_encoder_filename')
    if label_encoder_filename:
        label_encoder_path = os.path.join('/kaggle/input/input-model-sentiment/sentiment_analyzer_label_encoder.pkl')
        if os.path.exists(label_encoder_path):
            with open(label_encoder_path, 'rb') as f:
                label_encoder = pickle.load(f)
            print(f"Đã load label_encoder từ: {label_encoder_path}")
            print(f"Các lớp của label_encoder: {list(label_encoder.classes_)}")
        else:
            print(f"LỖI: Không tìm thấy file label_encoder tại {label_encoder_path}")
            label_encoder = None
    else:
        print("LỖI: Không có thông tin 'label_encoder_filename' trong model_info.")
        label_encoder = None
else:
    print(f"LỖI: Không tìm thấy model_info tại {model_info_path}")
    MAX_LEN = 128 
    label_encoder = None


phobert_tokenizer_name = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(phobert_tokenizer_name)
print(f"Đã load Tokenizer: {phobert_tokenizer_name}")

print("Đang tải PhoBERT encoder model (dùng để trích xuất embedding)...")
try:
    phobert_encoder = TFAutoModel.from_pretrained(phobert_tokenizer_name, from_pt=True)
    print("Đã tải xong PhoBERT encoder model.")
except Exception as e:
    print(f"Lỗi khi tải PhoBERT encoder model: {e}")
    phobert_encoder = None # Hoặc exit


def preprocess_text_for_prediction(text):
    text = str(text).lower()
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def predict_sentiment(text_input, keras_model, phobert_model_encoder, phobert_tokenizer, max_seq_len, lbl_encoder):
    if not all([keras_model, phobert_model_encoder, phobert_tokenizer, lbl_encoder]):
        print("Lỗi: Thiếu một trong các thành phần model hoặc tokenizer hoặc label encoder.")
        return None

    processed_text = preprocess_text_for_prediction(text_input)
    encoded_inputs = phobert_tokenizer(
        [processed_text],
        padding='max_length',
        truncation=True,
        max_length=max_seq_len,
        return_tensors='tf'
    )
    
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']

    physical_devices = tf.config.list_physical_devices('GPU')
    device_to_use = '/GPU:0' if physical_devices else '/CPU:0'
    
    with tf.device(device_to_use):
        phobert_outputs = phobert_model_encoder(input_ids=input_ids, attention_mask=attention_mask, training=False)
    
    input_embeddings_for_model = phobert_outputs.last_hidden_state 
    prediction_probabilities = keras_model.predict(input_embeddings_for_model)

    predicted_class_index = np.argmax(prediction_probabilities, axis=1)[0]
    predicted_label_text = lbl_encoder.inverse_transform([predicted_class_index])[0]
    confidence_score = float(prediction_probabilities[0][predicted_class_index])

    return {
        'text_input': text_input,
        'processed_text': processed_text,
        'label': predicted_label_text,
        'confidence': confidence_score,
        'prediction_vector': prediction_probabilities[0].tolist()
    }

if loaded_model and phobert_encoder and tokenizer and label_encoder and MAX_LEN:
    print("\n--- Bắt đầu dự đoán cho các câu test ---")
    test_texts = [
        "Sản phẩm này thực sự tuyệt vời, tôi rất hài lòng với chất lượng",
        "Dịch vụ khách hàng quá tệ, tôi sẽ không bao giờ quay lại nữa",
        "Món ăn bình thường, không có gì đặc biệt",
        "Quá thất vọng, không như quảng cáo",
        "Rất đáng tiền, sẽ mua lại ủng hộ shop!"
    ]

    for text_example in test_texts:
        result = predict_sentiment(
            text_example, 
            loaded_model, 
            phobert_encoder, 
            tokenizer, 
            MAX_LEN, 
            label_encoder
        )
        if result:
            print(f"\nVăn bản: {result['text_input']}")
            # print(f"Processed: {result['processed_text']}")
            print(f"  Nhãn dự đoán: {result['label']}")
            print(f"  Độ tin cậy: {result['confidence']:.2%}")
            # print(f"  Vector dự đoán: {result['prediction_vector']}")
            print("---")
else:
    print("\nKhông thể chạy dự đoán do thiếu model hoặc các thông tin cần thiết đã load bị lỗi.")



ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
import pickle
import os
import re

def preprocess_text_for_prediction(text):
    text = str(text).lower()
    text = re.sub(r'<[^>]+>', '', text) 
    text = re.sub(r'http\S+', '', text) 
    text = re.sub(r'\s+', ' ', text)    
    return text.strip()

def predict_sentiment(text_input, keras_model, phobert_model_encoder, phobert_tokenizer, max_seq_len, lbl_encoder):
    if not all([keras_model, phobert_model_encoder, phobert_tokenizer, lbl_encoder]):
        print("Lỗi trong predict_sentiment: Thiếu một trong các thành phần model, tokenizer, hoặc label encoder.")
        return None
    if max_seq_len is None:
        print("Lỗi trong predict_sentiment: max_seq_len is None.")
        return None

    processed_text = preprocess_text_for_prediction(text_input)
    encoded_inputs = phobert_tokenizer(
        [processed_text],      
        padding='max_length',
        truncation=True,
        max_length=max_seq_len,
        return_tensors='tf'    
    )
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']

    physical_devices = tf.config.list_physical_devices('GPU')
    device_to_use = '/GPU:0' if physical_devices else '/CPU:0'
    
    with tf.device(device_to_use):
        phobert_outputs = phobert_model_encoder(input_ids=input_ids, attention_mask=attention_mask, training=False)
    
    input_embeddings_for_model = phobert_outputs.last_hidden_state

    prediction_probabilities = keras_model.predict(input_embeddings_for_model, verbose=0) # Thêm verbose=0 để đỡ log

    predicted_class_index = np.argmax(prediction_probabilities, axis=1)[0]
    predicted_label_text = lbl_encoder.inverse_transform([predicted_class_index])[0]
    confidence_score = float(prediction_probabilities[0][predicted_class_index])

    return {
        'text_input': text_input,
        'processed_text': processed_text,
        'label': predicted_label_text,
        'confidence': confidence_score,
        'prediction_vector': prediction_probabilities[0].tolist()
    }

text_input_ui = widgets.Textarea(
    value='Sản phẩm này dùng cũng ổn, không quá tệ.',
    placeholder='Nhập văn bản cần phân tích...',
    description='Văn bản:',
    disabled=False,
    layout={'width': '95%', 'height': '100px'}
)

analyze_button_ui = widgets.Button(description="Phân tích thái độ", button_style='success', icon='search')
output_ui = widgets.Output()

def on_analyze_button_clicked(b):
    with output_ui:
        clear_output(wait=True)
        
        input_text_from_widget = text_input_ui.value.strip()
        
        if not input_text_from_widget:
            print("Vui lòng nhập văn bản!")
            return

        required_globals = ['loaded_model', 'phobert_encoder', 'tokenizer', 'MAX_LEN', 'label_encoder']
        missing_globals = [var_name for var_name in required_globals if var_name not in globals() or globals()[var_name] is None]
        
        if missing_globals:
            print(f"LỖI: Thiếu các biến global cần thiết để dự đoán: {', '.join(missing_globals)}")
            print("Vui lòng chạy lại các cell load model và thông tin ở trên trước.")
            return

        print(f"Đang phân tích: \"{input_text_from_widget[:100]}{'...' if len(input_text_from_widget) > 100 else ''}\"")

        result = predict_sentiment(
            input_text_from_widget,
            loaded_model,
            phobert_encoder,
            tokenizer,
            MAX_LEN,
            label_encoder
        )
        
        if result:
            print(f"\nNhãn dự đoán: {result['label']}")
            print(f"Độ tin cậy: {result['confidence']:.2%}")

            labels_plot = label_encoder.classes_
            prediction_vector_plot = result['prediction_vector']
            
            plt.figure(figsize=(8, 5))
            bars = plt.bar(labels_plot, prediction_vector_plot, color=['#FF9999','#ADD8E6','#90EE90']) # Màu cho NEG, NEU, POS
            
            for bar in bars:
                yval = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01, f'{yval:.2%}', ha='center', va='bottom')

            plt.title('Biểu đồ xác suất dự đoán cho từng nhãn', fontsize=14)
            plt.xlabel('Nhãn cảm xúc', fontsize=12)
            plt.ylabel('Xác suất', fontsize=12)
            plt.ylim(0, 1.05)
            plt.xticks(rotation=0, ha='center', fontsize=10)
            plt.yticks(fontsize=10)
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
        else:
            print("Không thể lấy được kết quả dự đoán. Vui lòng kiểm tra lại.")

analyze_button_ui.on_click(on_analyze_button_clicked)

print("--- Giao diện Phân tích Cảm xúc Văn bản ---")
display(widgets.VBox([text_input_ui, analyze_button_ui, output_ui]))



ModuleNotFoundError: No module named 'ipywidgets'